## Cross-Validation for Ridge Regression
Given the labeled training set $D_{train}=\{(x_i, y_i)\}_{i=1}^n$, where $y_i \in \mathbb R$, $x_i \in \mathbb R^d$, and $n \gg d$. This task is about cross-validation for ridge regression. Given $\lambda \in \{0.1, 1, 10, 100, 200\}$, we use $k$-Fold cross-validation technique to determine the best choice of $\lambda$ for the ridge regression problem. 

In other words, for each $\lambda$, we train a ridge regression $k=10$ times leaving out a different fold each time, and report the average of the RMSEs on the leaft-out folds. Then we choose the $\lambda$ that yields the smallest RMSE to train the model on the entire training set $D_{train}$. 

#### Imports & Reading Input

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error

df = pd.read_csv('train.csv')
y = df['y'].values
X = df.iloc[:, 1:].values
n, d = X.shape

#### Calculating the average RMSE
Given a trained model $w$ and data points $D_{train}$ consisting of $X_{train} \in \mathbb R^{n,d}$ and $y_{train} \in \mathbb R^n$, this function computes the training error. We define the training error as the root mean squared error (RMSE):$$\text{RMSE} = \sqrt{\frac 1 n \sum_{i=1}^n (y_i - w^Tx_i)^2}$$ Given a point $x_i\in \mathbb R^d$, we make the prediction $w^Tx_i = y_i$. Or equivalently, given a data matrix $X$, we make the prediction $Xw=y$. 

In [2]:
def calculate_RMSE(w, X_test, y_test):
    y_pred = X_test @ w
    RMSE = mean_squared_error(y_test, y_pred) ** 0.5
    return RMSE

#### Fitting the regressor
Given the training data $D_{train}$ consisting of $X_{train}$ and $y_{train}$, we fit the optimal regressor $\hat w$. Recall the ridge regression problem: $$\hat w = \arg \min_{w \in \mathbb R^d} \| y - Xw\|_2^2 + \lambda \|w\|_2^2$$The gradient of the loss function $L(w) = \|y - Xw \|_2^2 + \lambda \|w\|_2^2$ is $\triangledown L_w(w) = (X^TX + \lambda \text{Id})w - X^Ty$. Note that this problem provides a closed-form solution $\hat w = (X^TX + \lambda \text{Id})^{-1} X^Ty$, however, we use gradient descent with momentum for the sake of entertainment. The loss function $L$ is clearly convex, thus gradient descent will provide a global minimizer. Recall the update formula: $$w^{t+1} = w^t + \beta(w^{t-1} - w^t) - \alpha \triangledown L(w^t)$$

In [3]:
def fit(X_train, y_train, lam):
    alpha, beta = 1E-7, 0.01
    wt, wt1 = np.zeros(d), np.zeros(d) 
    Id = np.eye(d)

    for i in range(100): 
        grad = (X_train.T @ X_train + lam * Id) @ wt1 - X_train.T @ y_train
        wt2 = wt1 + beta * (wt - wt1) - alpha * grad
        wt = np.copy(wt1)
        wt1 = np.copy(wt2)

    w = wt2
    return w

#### K-fold Cross-Validation
We use the function `fit()` and `calculate_RMSE()` for the cross-validation loop. The idea of cross-validation is to train the model $\hat f_{\lambda_i}$ on $k=10$ different folds and to store the training error for each fold. That is, for each model $\hat f_{\lambda_i}$, average the training error over the 10 test folds, called cross-validation error of model $f_{\lambda_i}$, and save it. Finally, compare the cross-validation errors of all models $f_{\lambda_i}, i = 1,...5$, and choose the model with the smallest error. You can then train the model with the optimal $\lambda$ on the entire training set. 

In [4]:
lambdas = [0.1, 1, 10, 100, 200]
K = 10
RMSE_mat = np.zeros((K, len(lambdas)))
kf = KFold(n_splits=K)
i, j = 0, 0 

# train: indices of training data for current fold, e.g. i in train = i corresponds to i-th row of X_train, i.e. sample x_i of size d
# test:  indices of test data for current fold

for train, test in kf.split(X): 
    for lam in lambdas: 
        # training model 
        X_train, y_train = X[train], y[train]
        w = fit(X_train, y_train, lam)

        # testing model 
        X_test, y_test = X[test], y[test]
        RMSE_mat[i,j] = calculate_RMSE(w, X_test, y_test)
        j += 1
        
    i += 1 
    j = 0 
    

avg_RMSE = np.mean(RMSE_mat, axis=0) 

# figuring out the optimal lambda in lambdas
ind_min = np.argmin(avg_RMSE)
lam_opt = lambdas[ind_min]
print("Optimal Lambda:", lam_opt, "\n")

# train model with optimal lambda on entire data set
w_opt = fit(X, y, lam_opt)
print("Optimal w:", w_opt, "\n")
RMSE_opt = calculate_RMSE(w_opt, X, y)
print("Smallest RMSE:", RMSE_opt, "\n")

Optimal Lambda: 0.1 

Optimal w: [-3.12696226e+54 -5.78226440e+54 -7.48050724e+54 -4.63290931e+52
 -3.47164112e+53 -3.79754281e+54 -4.31605511e+55 -2.24367953e+54
 -7.07435227e+54 -2.79518966e+56 -1.14354973e+55 -2.12882551e+56
 -8.61248645e+54] 

Smallest RMSE: 2.0028980990287801e+59 

